In [122]:
import matplotlib.pyplot as plt
from matplotlib import style
import random
import numpy as np
import pandas as pd
from sklearn import datasets, svm, tree, preprocessing, metrics,linear_model
import sklearn.ensemble as ske
import tensorflow as tf
from sklearn.model_selection import train_test_split,ShuffleSplit,cross_validate
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import train_test_split

pd.options.display.float_format = '{:,.2f}'.format

df = pd.read_excel('titanic.xls', 'titanic3', index_col=None, na_values=['NA'])
df.fillna(0, inplace=True)

df = df.drop(['name','age','ticket','fare','cabin','embarked','home.dest','body'], axis =1)


le = LabelEncoder()
df["sex"] = le.fit_transform(df["sex"])

df.head()
# df.shape


,pclass,survived,sex,sibsp,parch,boat
0,1,1,0,0,0,2
1,1,1,1,1,2,11
2,1,0,0,1,2,0
3,1,0,1,1,2,0
4,1,0,0,1,2,0


In [115]:
def entropy(col):
    counts = np.unique(col,return_counts=True)
    ent = 0.0
    for ix in counts[1]:
        p = ix/col.shape[0]
        ent += (-1.0*p*np.log2(p))
    return ent

In [116]:

def divide_data(x_data,fkey,fval):
    x_right = pd.DataFrame([],columns=x_data.columns)
    x_left = pd.DataFrame([],columns=x_data.columns)
    for ix in range(x_data.shape[0]):
        val = x_data[fkey].loc[ix]
        if val >=fval:
            x_right = x_right.append(x_data.iloc[ix])
        else:
            x_left = x_left.append(x_data.iloc[ix])
    return x_right,x_left

In [117]:
def information_gain(x_data,fkey,fval):
    right,left = divide_data(x_data,fkey,fval)
    
    l = float(left.shape[0])/x_data.shape[0]
    r = float(right.shape[0])/x_data.shape[0]
    if left.shape[0] == 0 or right.shape[0] == 0:
        return -99999
    i_gain = entropy(x_data.output) - (l * entropy(left.output) + r*entropy(right.output))
    return i_gain

In [123]:
class DecisionTree:
    def __init__(self,depth=0,max_depth=5):
        self.left = None
        self.right = None
        self.fkey = None
        self.fval = None
        self.depth = depth
        self.max_depth = max_depth
        self.target = None
    def train(self,x_train):
        features = ['pclass', 'sex', 'age', 'sibSp', 'parch']
        info_gains = []
        for ix in features:
            i_gain = information_gain(x_train,ix,x_train[ix].mean())
            info_gains.append(i_gain)
        self.fkey = features[np.argmax(info_gains)]
        self.fval = x_train[self.fkey].mean()
        print("Splitting Tree",self.fkey)
        data_right,data_left = divide_data(x_train,self.fkey,self.fval)
        data_right = data_right.reset_index(drop=True)
        data_left = data_left.reset_index(drop=True)
        if data_left.shape[0] == 0 or data_right.shape[0] == 0:
            if x_train.output.mean() >= 0.5:
                self.target = "Positive"
            else:
                self.target = "Negative"
            return
        if self.depth >= self.max_depth:
            if x_train.output.mean() >= 0.5:
                self.target = "Positive"
            else:
                self.target = "Negative"
            return
        self.left = DecisionTree(self.depth+1,self.max_depth)
        self.left.train(data_left)
        self.right = DecisionTree(self.depth+1,self.max_depth)
        self.right.train(data_right)
        if x_train.output.mean() >= 0.5:
            self.target = "Positive"
        else:
            self.target = "Negative"
        return
    def predict(self,test):
        if test[self.fkey] > self.fval:
            if self.right is None:
                return self.target
            return self.right.predict(test)
        if test[self.fkey] < self.fval:
            if self.left is None:
                return self.target
            return self.left.predict(test)

In [124]:
split = int(0.7*df.shape[0])
train_data = df[:split]
test_data = df[split:]
test_data= test_data.reset_index(drop=True)

In [125]:

dt = DecisionTree()

In [127]:
dt.train(train_data)

AttributeError: 'DataFrame' object has no attribute 'output'

In [128]:
y_pred = []
for ix in range(test_data.shape[0]):
    y_pred.append(dt.predict(test_data.loc[ix]))

KeyError: None

In [129]:
y_pred[:10]

[]

In [130]:
for i in range(len(y_pred)):
    if y_pred[i] == "Negative":
        y_pred[i] = 0
    else:
        y_pred[i] = 1

In [131]:
np.mean(y_pred == test_data['output'])

KeyError: 'output'

In [132]:
dt.left.left.fkey

AttributeError: 'NoneType' object has no attribute 'left'

In [133]:
from sklearn.tree import DecisionTreeClassifier

In [134]:
features = ['pclass', 'sex', 'age', 'sibSp', 'parch']

In [135]:
sk_tree = DecisionTreeClassifier()

In [136]:
sk_tree.fit(train_data[features],train_data["output"])

KeyError: "['sibSp', 'age'] not in index"

In [137]:
sk_tree.predict(test_data[features])

KeyError: "['sibSp', 'age'] not in index"

In [138]:
sk_tree.score(test_data[features],test_data["output"])

KeyError: "['sibSp', 'age'] not in index"

In [139]:
from sklearn.ensemble import RandomForestClassifier

In [142]:
rf = RandomForestClassifier(n_estimators=10,criterion="entropy",max_depth=5)

In [143]:
rf.fit(train_data[features],train_data["output"])

KeyError: "['sibSp', 'age'] not in index"

In [144]:
rf.score(train_data[features],train_data["output"])

KeyError: "['sibSp', 'age'] not in index"

In [145]:
rf.score(test_data[features],test_data["output"])

KeyError: "['sibSp', 'age'] not in index"

In [ ]:
cross_val_score(RandomForestClassifier(n_estimators=10,criterion="entropy",max_depth=5),df[features],df["output"],cv=5).mean()

In [41]:
!pip3 install sklearn 

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
